In [ ]:
from NTDFacilitiesInventory import *
import constants
import folium
from shared_utils import catalog_utils
import datetime as dt

In [ ]:
catalog = catalog_utils.get_catalog("shared_data_catalog")
states = catalog.us_states.read()
california = states.loc[states.STATE_ABBR.isin(["CA"])].copy()
california

#### Run the below cell to get raw results from the NTD, geocode them, and save the results in GCS. This will be billed once Google Cloud free tier limits are met, you probably want to skip this cell if results have already been saved

In [ ]:
ntd_gdf = load_ntd(
    address_filter={ # this filter determines what addresses will be geocoded
        "State": "CA", 
        "City": "Los Angeles", 
        "Agency Name": "Southern California Regional Rail Authority"
    }, 
    geospatial_filter=california, # this filter determines which points will be saved
    attempt_geocoding=True
)

#### Run the below cell to load the saved results from GCS (skip if you ran the prior cell)

In [ ]:
# Load the NTD facilities inventory from the NTD (should not require geocoding)
ntd_gdf = load_ntd(
    url=constants.PROCESSED_GEOJSON_URI, attempt_geocoding=False,
)

#### Save the results as a shapefile and geojson in GCS

In [ ]:
facilities_inventory_gdf_to_shapefile(ntd_gdf, constants.PROCESSED_SHAPEFILE_URI)
facilities_inventory_gdf_to_geojson(
    ntd_gdf, constants.PROCESSED_GEOJSON_URI
)

#### Visualize the results in a Folium Map

In [ ]:
m = folium.Map(location=[36.8790073,-119.8346849], zoom_start=6)
facilities_inventory_gdf_to_folium(ntd_gdf).add_to(m)
m